<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/06_25_22_Fasttext_Embeddings_XGboost_Politifact_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U xgboost wandb fasttext pyfasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xgboost-1.6.1-py3-none-manylinux2014_x86_64.whl (192.9 MB)
  Using cached wandb-0.12.19-py2.py3-none-any.whl (1.8 MB)
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Using cached pyfasttext-0.4.6.tar.gz (244 kB)
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Using cached sentry_sdk-1.6.0-py2.py3-none-any.whl (145 kB)
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached setproctitle-1.2.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29 kB)
  Using cached shortuuid-1.0.9-py3-none-any.whl (9.4 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3146372 sha256=4e6b7204b4629ebe03e2a63242ab19ddc6d1ac7f1

In [6]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec, FastText
import fasttext as ft
import string
nltk.download('stopwords')
import re
import spacy
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb
from pathlib import Path

import wandb
from wandb.xgboost import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
base_dir = Path("/content/drive/MyDrive/")
assert base_dir.exists()

In [13]:
df = pd.read_csv(base_dir/"politifact_agg.csv", index_col=0)
df.head()

,title,text,tweets,retweets,label,url,tweet_ids,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Actress Emma Stone ‘For the first time in his...,NaN,[],"['1020554564334964741', '1020817527046197248',...",fake,NaN,[],2911,7.976595,0,0.0
1,Breaking President Trump makes English the of...,NaN,[],[],fake,NaN,[],0,0.000000,0,0.0
2,Friendly Fire … Charlottesville Car attacker ...,NaN,[],"['3265439004', '3250621593', '3253922920', '32...",fake,NaN,[],24,3.218876,0,0.0
3,If You Are Using This Toothpaste… Throw It Aw...,MightyLiving Blog\n\nHelpful inspiration from ...,[],"['911971426571255810', '1036749614853103616', ...",fake,mightynest.com/learn/research-by-concern/dange...,[],2569,7.851661,0,0.0
4,"""Face the Nation"" transcripts, August 26, 2012...","""Face the Nation"" transcripts, August 26, 2012...",[],[],real,https://web.archive.org/web/20120827001956/htt...,[],0,0.000000,0,0.0


In [14]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df["title"] = df.title.map(lambda x: ' '.join(simple_preprocess(x)))

In [15]:
# %%capture
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2022-06-25 10:46:13--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  63.8MB/s    in 86s     

2022-06-25 10:47:39 (50.1 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



trying to see similarity of vectors created.

In [16]:
# Loading model for English
modelog = ft.load_model('cc.en.300.bin')

In [17]:
ft_title_emb_file = base_dir/"ft_title_embeddings.npy"

if ft_title_emb_file.exists():
    title_embeddings = np.load(ft_title_emb_file)
else:
    title_embeddings = []
    for line in df['title']:
        emb = modelog.get_sentence_vector(line)
        title_embeddings.append(emb)
    title_embeddings = np.stack(title_embeddings)
    np.save(ft_title_emb_file, title_embeddings)

In [18]:
ft_text_emb_file = base_dir/"ft_text_embeddings.npy"

if ft_text_emb_file.exists():
    text_embeddings = np.load(ft_text_emb_file)
else:
    df.text.fillna("", inplace=True)
    texts = df.title + " " + df.text.map(lambda x: " ".join(simple_preprocess(x)))
    text_embeddings = []
    for line in texts:
        emb = modelog.get_sentence_vector(line)
        text_embeddings.append(emb)
    text_embeddings = np.stack(text_embeddings)
    np.save(ft_text_emb_file, text_embeddings)

In [19]:
import gc

del modelog; gc.collect()

580

### Title embeddings

In [20]:
X = title_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [21]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [22]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [23]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [25]:
GROUP = "fasttext-title-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

[0]	train-logloss:0.51851	eval-logloss:0.61572
[1]	train-logloss:0.40812	eval-logloss:0.58951
[2]	train-logloss:0.32666	eval-logloss:0.55907
[3]	train-logloss:0.26414	eval-logloss:0.53379
[4]	train-logloss:0.21567	eval-logloss:0.50541
[5]	train-logloss:0.18021	eval-logloss:0.49854
[6]	train-logloss:0.15367	eval-logloss:0.49576
[7]	train-logloss:0.13064	eval-logloss:0.48193
[8]	train-logloss:0.11177	eval-logloss:0.47289
[9]	train-logloss:0.09766	eval-logloss:0.47519
[10]	train-logloss:0.08434	eval-logloss:0.46956
[11]	train-logloss:0.07409	eval-logloss:0.46545
[12]	train-logloss:0.06489	eval-logloss:0.46792
[13]	train-logloss:0.05881	eval-logloss:0.46983
[14]	train-logloss:0.05286	eval-logloss:0.47294
[15]	train-logloss:0.04839	eval-logloss:0.47244
[16]	train-logloss:0.04411	eval-logloss:0.47571
[17]	train-logloss:0.04038	eval-logloss:0.47395
[18]	train-logloss:0.03704	eval-logloss:0.47463
[19]	train-logloss:0.03435	eval-logloss:0.48215
[20]	train-logloss:0.03209	eval-logloss:0.48404
[2

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▂▂▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
eval/accuracy,0.76966
eval/f1,0.78534
eval/precision,0.78947


### Text embeddings

In [26]:
X = text_embeddings
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((894, 300), (894,))

In [27]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [28]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [29]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [30]:
GROUP = "fastext-fulltext-xgb"
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="saloniteam", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

[0]	train-logloss:0.50363	eval-logloss:0.63250
[1]	train-logloss:0.38825	eval-logloss:0.59764
[2]	train-logloss:0.30897	eval-logloss:0.56504
[3]	train-logloss:0.24852	eval-logloss:0.55615
[4]	train-logloss:0.20233	eval-logloss:0.54239
[5]	train-logloss:0.16809	eval-logloss:0.51051
[6]	train-logloss:0.14092	eval-logloss:0.50119
[7]	train-logloss:0.11870	eval-logloss:0.49726
[8]	train-logloss:0.10371	eval-logloss:0.48790
[9]	train-logloss:0.08863	eval-logloss:0.48295
[10]	train-logloss:0.07685	eval-logloss:0.47724
[11]	train-logloss:0.06773	eval-logloss:0.47921
[12]	train-logloss:0.06026	eval-logloss:0.47675
[13]	train-logloss:0.05284	eval-logloss:0.46977
[14]	train-logloss:0.04717	eval-logloss:0.46967
[15]	train-logloss:0.04296	eval-logloss:0.46174
[16]	train-logloss:0.03926	eval-logloss:0.46636
[17]	train-logloss:0.03591	eval-logloss:0.46721
[18]	train-logloss:0.03309	eval-logloss:0.47220
[19]	train-logloss:0.03086	eval-logloss:0.47122
[20]	train-logloss:0.02891	eval-logloss:0.48094
[2

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▂▁▁▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
eval/accuracy,0.82584
eval/f1,0.84103
eval/precision,0.82828
